In [1]:
from typing import Union
from pathlib import Path
import datetime
import logging
import psycopg
import db
import sec_financials
from sqlalchemy.inspection import inspect
from psycopg import sql

In [2]:
logging.basicConfig(level=logging.INFO)

In [2]:
db.models.Num.__table__.drop(db.setup.engine)
db.models.Pre.__table__.drop(db.setup.engine)
db.models.Tag.__table__.drop(db.setup.engine)
db.models.Sub.__table__.drop(db.setup.engine)

In [3]:
db.setup.main_setup()
sec = sec_financials.SecFiles(start_date='2010-01-01')

In [13]:
with psycopg.connect(db.settings.Settings().conn_str) as conn:
    with conn.cursor() as cur:
        #pass
        logging.info(f"Filling Sub Table")
        db.ops.copy_to_table(cur, db.models.Sub, sec.sub)
        logging.info(f"Filling Sub Table")
        db.ops.insert_records(db.models.Tag, sec.tag, True)
        logging.info(f"Filling Sub Table")
        db.ops.copy_to_table(cur, db.models.Num, sec.num)
        logging.info(f"Filling Sub Table")
        db.ops.copy_to_table(cur, db.models.Pre, sec.pre)

INFO:root:COPY "sub" ("adsh","cik","name","sic","countryba","stprba","cityba","zipba","bas1","bas2","baph","countryinc","stprinc","ein","former","changed","afs","wksi","fye","form","period","fy","fp","filed","accepted","prevrpt","detail","instance","nciks","aciks") FROM STDIN
INFO:root:<psycopg.Copy [ACTIVE] (host=localhost user=root database=postgres) at 0x7f11e8463940>


In [4]:
db.ops.insert_records(db.models.Sub, sec.sub, True)

In [8]:
db.ops.insert_records(db.models.Sub, sec.sub)

INFO:root:Using standard sqlalchemy method


In [4]:
i = 0
for row in sec.pre:
    i+=1
    x = db.models.Pre.parse_obj(row)

print(i)

46675423


In [34]:
def yr_qtr_to_date(yr_qtr_str: str) -> datetime.date:
    """ Converts a str in the form {year}Q{qtr} (ie 2010Q2) to the end of the quarter 2010-03-31 """
    yr, qtr = yr_qtr_str.split("Q")
    mon = 1 + int(qtr)*3 if int(qtr) < 4 else 1
    yr = int(yr) if mon > 1 else int(yr) + 1
    return datetime.date(yr, mon, 1) - datetime.timedelta(days=1)
sorted([(yr_qtr_to_date(f.name),f) for f in list(Path('./files').iterdir())], key=lambda x: x[0])

[(datetime.date(2009, 3, 31), PosixPath('files/2009Q1')),
 (datetime.date(2009, 6, 30), PosixPath('files/2009Q2')),
 (datetime.date(2009, 9, 30), PosixPath('files/2009Q3')),
 (datetime.date(2009, 12, 31), PosixPath('files/2009Q4')),
 (datetime.date(2010, 3, 31), PosixPath('files/2010Q1')),
 (datetime.date(2010, 6, 30), PosixPath('files/2010Q2')),
 (datetime.date(2010, 9, 30), PosixPath('files/2010Q3')),
 (datetime.date(2010, 12, 31), PosixPath('files/2010Q4')),
 (datetime.date(2011, 3, 31), PosixPath('files/2011Q1')),
 (datetime.date(2011, 6, 30), PosixPath('files/2011Q2')),
 (datetime.date(2011, 9, 30), PosixPath('files/2011Q3')),
 (datetime.date(2011, 12, 31), PosixPath('files/2011Q4')),
 (datetime.date(2012, 3, 31), PosixPath('files/2012Q1')),
 (datetime.date(2012, 6, 30), PosixPath('files/2012Q2')),
 (datetime.date(2012, 9, 30), PosixPath('files/2012Q3')),
 (datetime.date(2012, 12, 31), PosixPath('files/2012Q4')),
 (datetime.date(2013, 3, 31), PosixPath('files/2013Q1')),
 (datetime

In [4]:
list(sec.num)

INFO:root:Skipping quarter ending 2009-03-31
INFO:root:Skipping quarter ending 2009-06-30
INFO:root:Skipping quarter ending 2009-09-30
INFO:root:Skipping quarter ending 2009-12-31
INFO:root:Reading file from quarter ending 2010-03-31
INFO:root:Reading file from quarter ending 2010-06-30
INFO:root:Reading file from quarter ending 2010-09-30
INFO:root:Reading file from quarter ending 2010-12-31
INFO:root:Reading file from quarter ending 2011-03-31


KeyboardInterrupt: 

In [11]:
db.models.Sub.__table__.insert().values({})

In [12]:
from sqlalchemy.dialects.postgresql import insert

In [14]:
insert(db.models.Sub).on_conflict_do_nothing()